In [1]:
import requests
import pandas as pd

# API requests
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')

# Builiding the Dataframes
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)

# Merging the Tables
complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [3]:
years = ['2014', '2015', '2016',]
categories = ['A', 'B', 'C', 'D']
plane_dimensions = ['v4', 'v5']
size_dimension = 'v3'


custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}

rdf = complete_reports_df
mdf = complete_measurements_df

# columns = []

grid_data = {}

for year in years:
    for category in categories:
        filtered_rdf = rdf[(rdf.year == int(year)) & (rdf.category == category)]
        filtered_mdf = mdf[(mdf.year == int(year)) & (mdf.category == category)]
        
        for dim in plane_dimensions:
            column_name = f'{year}_{category}_{dim}'
            grid_data[column_name] = list(filtered_rdf[dim])
             
        grid_data[f'{year}_{category}_{size_dimension}'] = list(filtered_rdf[size_dimension])
        grid_data[f'{year}_{category}_codes'] = list(filtered_rdf['code'])
#             columns.append(column_name)
#             column = Column(column_data, column_name)
#             columns.append(column)
        
# pandas_grid = pd.DataFrame(pandas_grid_data, columns=columns)

In [5]:
variables_r = requests.get('https://peridash.ml/api/variables')
vdf = pd.read_json(variables_r.text)

In [39]:
import numpy as np
import math

figure = {
    'data': [],
    'frames': [],
    'layout': {},
}

custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}


for category in categories:
    data_dict = {
        'x': grid_data[f'2014_{category}_{plane_dimensions[0]}'],
        'y': grid_data[f'2014_{category}_{plane_dimensions[1]}'],
        'mode': 'markers',
        'marker': {
            'sizemode': 'area',
            'sizeref': 10000,
            'size': [0.0 if math.isnan(x) else x for x in grid_data[f'{year}_{category}_{size_dimension}']],
            'color': custom_colors[category],
        },
        'text': grid_data[f'2014_{category}_codes'],
        'name': 'Categoría: ' + category,
    }
    figure['data'].append(data_dict)
    
def create_frame(year):
    frame_data = []
    for category in categories:
        year_cat_dict = {
            'x': grid_data[f'{year}_{category}_{plane_dimensions[0]}'],
            'y': grid_data[f'{year}_{category}_{plane_dimensions[1]}'],
            'mode': 'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 10000,
                'size': [0.0 if math.isnan(x) else x for x in grid_data[f'{year}_{category}_{size_dimension}']],
                'color': custom_colors[category],
            },
            'text': grid_data[f'{year}_{category}_codes'],
            'name': 'Categoría: ' + category,
        }
        frame_data.append(year_cat_dict)
        
    return dict(data=frame_data, name=year)

figure['frames'] = [create_frame(year) for year in years]

animation_settings = dict(
    frame={'duration': 700, 'redraw': False},
    fromcurrent=False,
    transition={'duration': 700, 'easing':'cubic-in-out'},
)

steps = []
for year in years:
    step = dict(
        method = 'animate',  
        args = [[year], animation_settings],
        label= year
    )
    steps.append(step)

figure['layout']['sliders'] = [dict(
    active = 0,
    currentvalue = {
        'prefix': 'Año: ',
        'font': {'size': 20},
        'visible': True,
        'xanchor': 'right'
    },
    steps = steps,
    yanchor= 'top',
    xanchor= 'left',
    pad= {'b': 10, 't': 50},
    len= 0.9,
    x= 0.1,
    y= 0,
)]

figure['layout']['updatemenus'] = [{
    'buttons': [{
        'args': [years, animation_settings],
        'label': 'Animar',
        'method': 'animate',
    }],
    'direction': 'left',
    'pad': {'r': 10, 't': 87},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
}]

x_ind = plane_dimensions[0][1:]
y_ind = plane_dimensions[1][1:]

figure['layout']['hovermode'] = 'closest'
figure['layout']['xaxis'] = {'title': f'Variable {x_ind}: {vdf[vdf.var_id==int(x_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(x_ind)]["unit"].iloc[0]})'}
figure['layout']['yaxis'] = {'title': f'Variable {y_ind}: {vdf[vdf.var_id==int(y_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(y_ind)]["unit"].iloc[0]})'}
figure['layout']['height'] = 700
figure['layout']['width'] = 1000
figure['layout']['plot_bgcolor'] = '#dfe8f3'
figure['layout']['dragmode'] = 'lasso'
figure['layout']['title'] = 'Variables a través de los años'

In [40]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
iplot(figure)

In [43]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
import math

years = ['2014', '2015', '2016',]
categories = ['A', 'B', 'C', 'D']
plane_dimensions = ['v4', 'v5']
size_dimension = 'v3'

custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}

rdf = complete_reports_df
mdf = complete_measurements_df

columns = []
grid_data = {}

for year in years:
    for category in categories:
        filtered_rdf = rdf[(rdf.year == int(year)) & (rdf.category == category)]
#         filtered_mdf = mdf[(mdf.year == int(year)) & (mdf.category == category)]
        
        for dim in plane_dimensions:
            column_name = f'{year}_{category}_{dim}'
            column_data = list(filtered_rdf[dim])
            column = Column(column_data, column_name)
            columns.append(column)
            grid_data[column_name] = list(filtered_rdf[dim])
             
        
        column_name = f'{year}_{category}_{size_dimension}'
        column_data = list(filtered_rdf[size_dimension])
        clean_column_data = [0.0 if math.isnan(x) else x for x in column_data]
        column = Column(clean_column_data, column_name)
        columns.append(column)
        
        column_name = f'{year}_{category}_codes'
        column_data = list(filtered_rdf['code'])
        column = Column(column_data, column_name)
        columns.append(column)
        
grid = Grid(columns)
py.grid_ops.upload(grid, 'animated_variables_grid_new')

'https://plot.ly/~sergiochumacero/63/'

In [44]:
import numpy as np

figure = {
    'data': [],
    'frames': [],
    'layout': {},
}

custom_colors = {
    'A': 'rgb(171, 99, 250)',
    'B': 'rgb(230, 99, 250)',
    'C': 'rgb(99, 110, 250)',
    'D': 'rgb(25, 211, 243)',
#     'Oceania': 'rgb(50, 170, 255)'
}


for category in categories:
    data_dict = {
        'xsrc': grid.get_column_reference(f'2014_{category}_{plane_dimensions[0]}'),
        'ysrc': grid.get_column_reference(f'2014_{category}_{plane_dimensions[1]}'),
        'mode': 'markers',
        'marker': {
            'sizemode': 'area',
            'sizeref': 10000,
            'sizesrc': grid.get_column_reference(f'2014_{category}_{size_dimension}'),
            'color': custom_colors[category],
        },
        'textsrc': grid.get_column_reference(f'2014_{category}_codes'),
        'name': 'Categoría: ' + category,
    }
    figure['data'].append(data_dict)
    
def create_frame(year):
    frame_data = []
    for category in categories:
        year_cat_dict = {
            'xsrc': grid.get_column_reference(f'{year}_{category}_{plane_dimensions[0]}'),
            'ysrc': grid.get_column_reference(f'{year}_{category}_{plane_dimensions[1]}'),
            'mode': 'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 10000,
                'sizesrc': grid.get_column_reference(f'{year}_{category}_{size_dimension}'),
                'color': custom_colors[category],
            },
            'textsrc': grid.get_column_reference(f'{year}_{category}_codes'),
            'name': 'Categoría: ' + category,
        }
        frame_data.append(year_cat_dict)
        
    return dict(data=frame_data, name=year)

figure['frames'] = [create_frame(year) for year in years]

animation_settings = dict(
    frame={'duration': 700, 'redraw': False},
    fromcurrent=False,
    transition={'duration': 700, 'easing':'cubic-in-out'},
)

steps = []
for year in years:
    step = dict(
        method = 'animate',  
        args = [[year], animation_settings],
        label= year
    )
    steps.append(step)

figure['layout']['sliders'] = [dict(
    active = 0,
    currentvalue = {
        'prefix': 'Año: ',
        'font': {'size': 20},
        'visible': True,
        'xanchor': 'right'
    },
    steps = steps,
    yanchor= 'top',
    xanchor= 'left',
    pad= {'b': 10, 't': 50},
    len= 0.9,
    x= 0.1,
    y= 0,
)]

figure['layout']['updatemenus'] = [{
    'buttons': [{
        'args': [years, animation_settings],
        'label': 'Animar',
        'method': 'animate',
    }],
    'direction': 'left',
    'pad': {'r': 10, 't': 87},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
}]

x_ind = plane_dimensions[0][1:]
y_ind = plane_dimensions[1][1:]

figure['layout']['hovermode'] = 'closest'
figure['layout']['xaxis'] = {'title': f'Variable {x_ind}: {vdf[vdf.var_id==int(x_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(x_ind)]["unit"].iloc[0]})'}
figure['layout']['yaxis'] = {'title': f'Variable {y_ind}: {vdf[vdf.var_id==int(y_ind)]["name"].iloc[0]} ({vdf[vdf.var_id==int(y_ind)]["unit"].iloc[0]})'}
figure['layout']['height'] = 700
figure['layout']['width'] = 1000
figure['layout']['plot_bgcolor'] = '#dfe8f3'
figure['layout']['dragmode'] = 'lasso'
figure['layout']['title'] = 'Variables a través de los años'

In [45]:
import plotly.plotly as py
py.icreate_animations(figure, filename='animated_variables_new')